[개념정리]
=============


In [ ]:
# pg.201 : human_activity 데이터 불러와서 전처리하기

import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

# features.txt 파일에는 피처 이름 index와 피처명이 공백으로 분리되어 있음. 이를 DataFrmame으로 로드.
feature_name_df = pd.read_csv('/content/sample_data/features.txt', sep='\s+',
                             header=None, names=['column_index','column_name'])

# 피처명 index를 제거하고, 피처명과 리스트 객체로 생성한 뒤 샘플로 10개만 추출
feature_name = feature_name_df.iloc[:,1].values.tolist()
print('전체 피처명에서 10개만 추출:', feature_name[:10])

In [ ]:
# pg.203 : 중복된 피처명이 얼마나 있는지 알아보기

feature_dup_df = feature_name_df.groupby('column_name').count()
print(feature_dup_df[feature_dup_df['column_index'] > 1].count())
feature_dup_df[feature_dup_df['column_index'] > 1].head()

In [ ]:
# pg.204 : 중복된 피처명에 대해 원본 피처명에 _1 또는 _2을 추가로 부여해 새로운 피처명 가지는 df 반환하는 함수 생성

def get_new_feature_name_df(old_feature_name_df):
    feature_dup_df = pd.DataFrame(data=old_feature_name_df.groupby('column_name').cumcount(), columns=['dup_cnt'])
    feature_dup_df = feature_dup_df.reset_index()
    new_feature_name_df = pd.merge(old_feature_name_df.reset_index(), feature_dup_df, how='outer')
    new_feature_name_df['column_name'] = new_feature_name_df[['column_name','dup_cnt']].apply(lambda x: x[0]+'_'+str(x[1]) if x[1]>0 else x[0], axis=1)
    new_feature_name_df = new_feature_name_df.drop(['index'], axis=1)
    return new_feature_name_df

In [ ]:
# pg204 : human_activity 데이터를 나타내는 df를 생성하는 로직을 간단한 함수로 생성하기

import pandas as pd

def get_human_dataset():
    
    # 각 데이터 파일은 공백으로 분리되어 있으므로 read_csv에서 공백 문자를 sep으로 할당.
    feature_name_df = pd.read_csv('/content/sample_data/features.txt', sep='\s+',header=None, names=['column_index','column_name'])
                                  
    # 중복된 피처명을 수정하는 get_new_feature_name_df()를 이용, 신규 피처명 DataFrame 생성
    new_feature_name_df = get_new_feature_name_df(feature_name_df)
    
    # DataFrame에 피처명을 칼럼으로 부여하기 위해 리스트 객체로 다시 변환
    feature_name = new_feature_name_df.iloc[:,1].values.tolist()
                                  
    # 학습 피처 데이터세트와 테스트 피처 데이터를 DataFrame으로 로딩. 칼럼명은 feature_name 적용
    X_train = pd.read_csv('/content/sample_data/X_train.txt',sep='\s+', names= feature_name)
    X_test = pd.read_csv('/content/sample_data/X_test.txt',sep='\s+', names= feature_name)
                                  
    # 학습 레이블과 테스트 레이블 데이터를 DataFrame으로 로딩하고 칼럼명은 action으로 부여
    y_train = pd.read_csv('/content/sample_data/y_train.txt',sep='\s+', header=None,names= ['action'])
    y_test = pd.read_csv('/content/sample_data/y_test.txt',sep='\s+', header=None,names= ['action'])
                                  
    # 로드된 학습/테스트용 DataFrame을 모두 반환
    return X_train, X_test, y_train, y_test
                                  
X_train, X_test, y_train, y_test = get_human_dataset()

In [ ]:
# pg.205 : 로드한 학습용 피처 데이터 세트 살펴보기

print('## 학습 피처 데이터셋 info()')
print(X_train.info())

- 학습 데이터 세트 : 7352개의 레코드, 561개의 피처
- 피처가 모두 float 형의 숫자형 -> 별도의 인코딩이 필요하지 않음

In [ ]:
print(y_train['action'].value_counts())

- 레이블 값은 6개 값으로 비교적 고르게 분포되어 있음

In [ ]:
# pg.206 : DecisionTreeClassifier 이용해서 동작 예측 분류 수행하기

from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

# 예제 반복 시마다 동일한 예측 결과 도출을 위해 random_state 설정
dt_clf = DecisionTreeClassifier(random_state=156)
dt_clf.fit(X_train, y_train)
pred = dt_clf.predict(X_test)
accuracy = accuracy_score(y_test, pred)
print('결정 트리 예측 정확도: {0:.4f}'.format(accuracy))

# DecisionTreeClassifier의 하이퍼 파라미터 추출
print('DecisionTreeClassifier 기본 하이퍼 파라미터:\n', dt_clf.get_params())

In [ ]:
# pg.207 : 결정 트리의 트리 깊이가 예측 정확도에 주는 영향 살펴보기 -> GridSearchCV 이용해서 ~

from sklearn.model_selection import GridSearchCV

params = {
    'max_depth' : [6,8,10,12,16,20,24]
}

grid_cv = GridSearchCV(dt_clf, param_grid=params, scoring='accuracy', cv=5, verbose=1)
grid_cv.fit(X_train, y_train)
print("GridSearchCV 최고 평균 정확도 수치: {0:.4f}".format(grid_cv.best_score_))
print("GridSearchCV 최적 하이퍼 파라미터:", grid_cv.best_params_)

max_depth가 8일 때 5개의 폴드 세트의 최고 평균 정확도가 85.26%로 도출됨

In [ ]:
# pg.208 : GridSearchCV 객체의 cv_results_ 속성 이용해서 max_depth 값에 따라 예측 성능이 어떻게 변하는지 보기

# GridSearchCV 객체의 cv_results_ 속성을 DataFrame으로 생성.
cv_results_df = pd.DataFrame(grid_cv.cv_results_)

# max_depth 파라미터 값과 그때의 테스트 세트, 학습 데이터 세트의 정확도 수치 추출
cv_results_df[['param_max_depth', 'mean_test_score']]

mean_test_score : 5개의 CV 세트에서 검증용 데이터 세트의 정확도 평균 수치
- max_depth가 8일 때 0.852로 정확도가 정점이고 넘어서면 점점 정확도가 떨어짐

In [ ]:
# pg.209 : 별도의 테스트 데이터에서 결정 트리의 정확도 측정해보기

max_depths = [ 6, 8, 10, 12, 16, 20, 24]
# max_depth 값을 변화시키면서 그때마다 학습과 테스트 세트에서의 예측 성능 측정
for depth in max_depths:
    dt_clf = DecisionTreeClassifier(max_depth=depth, random_state=156)
    dt_clf.fit(X_train, y_train)
    pred = dt_clf.predict(X_test)
    accuracy = accuracy_score(y_test, pred)
    print('max_depth = {0} 정확도: {1:.4f}'.format(depth, accuracy))

- max_depth가 8일 때 87.07%로 가장 높고 8을 넘어설수록 정확도는 계속 감소함
- 결정 트리는 깊이가 깊어질수록 과적합의 영향력이 커지므로 하이퍼 파라미터를 이용해 깊이를 제어해야 함

In [ ]:
# pg.210 : max_depth와 min_samples_split을 같이 변경하면서 정확도 성능 튜닝하기

params = {
    'max_depth' : [8, 12, 16, 20],
    'min_samples_split' : [16, 24],
}

grid_cv = GridSearchCV(dt_clf, param_grid=params, scoring='accuracy', cv=5, verbose=1)
grid_cv.fit(X_train, y_train)
print("GridSearchCV 최고 평균 정확도 수치: {0:.4f}".format(grid_cv.best_score_))
print("GridSearchCV 최적 하이퍼 파라미터:", grid_cv.best_params_)

grid_cv의 속성인 best_estimator_는 최적 하이퍼 파라미터인 max_depth 8, min_samples_split 16으로 학습이 완료된 Estimator 객체임

In [ ]:
# pg.210 : 최적 하이퍼 파라미터를 이용해서 테스트 데이터 세트에 예측 수행하기

best_df_clf = grid_cv.best_estimator_
pred1 = best_df_clf.predict(X_test)
accuracy = accuracy_score(y_test, pred1)
print('결정 트리 예측 정확도:{0:.4f}'.format(accuracy))

In [ ]:
# pg.210 : 결정 트리에서 각 피처의 중요도를 feature_importances_ 속성 이용해서 알아보기
# (중요도가 높은 순으로 top 20 피처를 막대그래프로 표현하기)

import seaborn as sns

ftr_importances_values = best_df_clf.feature_importances_
# Top 중요도로 정렬을 쉽게 하고, 시본(Seaborn)의 막대그래프로 쉽게 표현하기 위해 Series 변환
ftr_importances = pd.Series(ftr_importances_values, index=X_train.columns)
# 중요도값 순으로 Series를 정렬
ftr_top20 = ftr_importances.sort_values(ascending=False)[:20]
plt.figure(figsize=(8,6))
plt.title("Feature importances Top 20")
sns.barplot(x = ftr_top20, y = ftr_top20.index)
plt.show()